In [19]:
import torch
from torch.utils.data import Dataset, DataLoader

from skimage import io

import numpy as np
import torch
import os
from PIL import Image, ImageOps, ImageFilter
import random


In [20]:
def data_generator(original_data_folder, augmented_data_folder, city, amount):
    
    path = os.path.join(original_data_folder, city)
    count = 0
    for imageFile in os.listdir(path):
        imagePath = os.path.join(path, imageFile).replace("\\", "/")
        
        # Transform image - slight rotations and 
        image = Image.open(imagePath)
        
        rand = random.randint(0,9)
        if rand <= 2:
            rand2= random.randint(-3,3)
            image = image.rotate(rand2)
        elif rand <= 5:
            image = ImageOps.grayscale(image)
        else:
            image = image.filter(ImageFilter.GaussianBlur(1))
        
        
        #Write Image to AugmentedData
        image.save(os.path.join(os.path.join(augmented_data_folder, city), str(count) + ".0,1.0.jpg").replace("\\", "/"))
       
        count += 1
        if count >= amount:
            break

In [21]:
data_generator("./Data/", "./AugmentedData/", "Atlanta", 10)

In [ ]:
import CitiesData
from torchvision.transforms import transforms
batch_size = 32
transform = transforms.Compose([transforms.ToTensor(), transforms.RandomResizedCrop(size=(224, 224), antialias=True), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainDataLoader, testDataLoader = CitiesData.getBalancedCitiesDataLoader("./Data/", "./AugmentedData/", transforms = transform, batchSize=batch_size)